In [3]:
import os
from glob import glob
label_dir = '/root/autodl-tmp/github/yolov10/runs/detect/output2/labels'
import json

"""
    {
      "frame_id": 20,
      "event_id": 1,
      "category": "机动车违停",
      "bbox": [200, 300, 280, 500],
      "confidence": 0.85
    },
"""
category_cn_list = ['机动车违停', '非机动车违停', '违法经营', '垃圾桶满溢']
width = 1920
height = 1080
for video_index in range(39):
    video_json= []
    video_labels = sorted(glob(f'{label_dir}/test_{video_index:03d}*.txt'))
    for label_file in video_labels:
        label_txt = os.path.basename(label_file).replace('.txt', '')
        str_split = label_txt.split('_')
        assert(len(str_split)) == 3, str_split
        frame_id = int(str_split[2])
        event_id = 1
        with open(label_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            result = line.strip().split(' ')
            assert(len(result)) == 6, result
            cat_id, conf, cx, cy, w, h = int(result[0]), float(result[1]), float(result[2]), float(result[3]),  float(result[4]), float(result[5])
            cat_name = category_cn_list[int(cat_id)]
            x1 = int(max((cx-w/2), 0)*width)
            x2 = int(min((cx+w/2), 1)*width)
            y1 = int(max((cy-h/2), 0)*height)
            y2 = int(min((cy+h/2), 1)*height)
            video_json.append({
                "frame_id": frame_id,
                "event_id": event_id,
                "category": cat_name,
                "bbox": [x1, y1, x2, y2],
                "confidence": conf
            })
            event_id = event_id + 1
    json_string = json.dumps(video_json, indent=4)
    json_file = f'test_{video_index}.json'
    with open(f'/root/ws/2024-mars/result/{json_file}', 'w') as f:
        f.write(json_string)

